In [ ]:
!pip install news-please

In [3]:
import csv
import pandas as pd
import pickle
from itertools import accumulate, chain, repeat, tee
import difflib
import matplotlib
from newsplease import NewsPlease

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
filepath = '/content/drive/MyDrive/Project/VIP'

data = pd.read_csv('/content/drive/MyDrive/Project/VIP/allsides-content-f.csv')
    
data.head()

,Date,Description,Source_Name,Source_Bias,Source_Headline,Source_Link
0,2023-03-09T08:06:49-08:00,"For more than two years, while they controlled...",OPINION: Trump’s risky call for protests,Left,https://www.allsides.com/news/2023-03-09-0806/...,https://nypost.com/2023/03/08/an-egregious-den...
1,2020-11-04T21:50:01-08:00,When Donald Trump narrowly won Wisconsin in 20...,OPINION: Trump’s risky call for protests,Left,https://www.allsides.com/news/2020-11-04-2150/...,https://www.washingtonpost.com/politics/americ...
2,2022-07-20T12:44:42-07:00,Joe Biden has won the U.S. presidential electi...,OPINION: Donald Trump has committed a lot of s...,Lean Left,https://www.allsides.com/news/2020-11-10-1027/...,https://www.niemanlab.org/2020/11/five-types-o...
3,2020-12-16T11:02:25-08:00,WASHINGTON – Congressional Republicans began t...,Trump Indictment Could Be a 2024 Cash Cow,Center,https://www.allsides.com/news/2020-12-16-1101/...,https://www.usatoday.com/story/news/politics/e...
4,2020-11-09T19:10:07-08:00,U.S. Attorney General William Barr has authori...,Police surround NY Courthouse and DC Capitol i...,Center,https://www.allsides.com/news/2020-11-09-1910/...,https://www.newsweek.com/barr-tells-federal-pr...


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113 entries, 0 to 112
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Date             113 non-null    object
 1   Description      113 non-null    object
 2   Source_Name      113 non-null    object
 3   Source_Bias      113 non-null    object
 4   Source_Headline  113 non-null    object
 5   Source_Link      113 non-null    object
dtypes: object(6)
memory usage: 5.4+ KB


In [ ]:
data['Source_Link'].values.tolist()

In [25]:
data['Source_Link'] = data['Source_Link'].apply(lambda link: link.rstrip('"'))

In [ ]:
news_links = [link for link in data['Source_Link']]
news_links

Next, we are going to compile the list of URLs into chunks. To do this, we're going to create a function that accepts two arguments: the list of URLs and the number of chunks that we want to create.

Assuming that the number of chucks is greater than zero, the function is going to measure the size of the list and divide it by the number of chunks we are trying to create.

In [ ]:
import math
def chuck(xs, n):
    assert n > 0
    L = len(xs)
    s, r = divmod(L, n)
    widths = chain(repeat(s+1, r), repeat(s, n-r))
    offsets = accumulate(chain((0,), widths))
    b, e = tee(offsets)
    next(e)
    return [xs[s] for s in map(slice, b, e)]


batch = chuck(news_links, 60)

batch[3:7]

In [ ]:
batch

In [9]:
def article_crawler():
    # crawler
    n = 0
    for i in range(0, len(batch)):
        try:
            slice = batch[i]
            # print slice
            slice_name = str(i) + '-NewsPlease-articleCrawl.p'
            article_information = NewsPlease.from_urls(slice)
            pickle.dump(article_information, open(slice_name, 'wb'))
            n += 1
        except:
            continue
            
article_crawler()

ERROR:newsplease.crawler.simple_crawler:not a 200 response: 400
ERROR:newsplease.crawler.simple_crawler:not a 200 response: 403
ERROR:newsplease.crawler.simple_crawler:not a 200 response: 403
ERROR:newsplease.crawler.simple_crawler:not a 200 response: 403


In [10]:
# helper function
def make_unique(url_list):
   # Not order preserving    
   unique = set(url_list)
   return list(unique)

def check_data(filepath):
    scraped = []
    not_scraped = []

    for i in range(0, (math.ceil(len(news_links)/2))):
        try:
            #file_path = filepath+"/crawl/"
            open_crawl = pickle.load(open(str(i) + "-NewsPlease-articleCrawl.p", "rb"))
            for url in open_crawl:
                text = open_crawl[str(url)].maintext
                if text == None:
                    not_scraped.append(url)
                else:
                    scraped.append(url)
        except FileNotFoundError:
            continue

    scraped = make_unique(scraped)
    return scraped, not_scraped
            
success, fail = check_data(filepath)

# analyze data collection success
def percentage(part, whole):
    percent = 100 * float(part)/float(whole)
    format = "{0:.2f}".format(percent)
    return format+'%'

print("The extraction process yielded " 
      + str(len(success)) + " articles, or "
      + percentage(len(success),len(news_links)) 
      + " of the total.")

The extraction process yielded 95 articles, or 84.07% of the total.


In [12]:
def get_data():
    news_dict = {}

    remove_list = ['www.', 'www1.' '.com', '.gov', '.org', 'beta.', ',eu',
                   '.co.uk', 'europe', 'gma', 'blogs', 'in.', 'm.',
                   'eclipse2017.', 'money', 'insider', 'news.', 'finance.'
                   'www1.']

    for i in range(0, 220):
        try:
            #file_path = filepath + "/crawl/"
            open_crawl = pickle.load(open(str(i) + "-NewsPlease-articleCrawl.p", "rb"))

            for url in open_crawl:
                text = open_crawl[str(url)].maintext
                if text != None:
                    title = open_crawl[str(url)].title
                    authors = ', '.join(open_crawl[str(url)].authors)
                    source = open_crawl[str(url)].source_domain

                    for seq in remove_list:
                        if seq in source:
                            source = source.replace(seq, "")

                    date = open_crawl[str(url)].date_publish

                    news_dict[str(url)] = [source, title, authors, date, text]

        except FileNotFoundError:
            continue

    return news_dict


all_news = get_data()
print(all_news)

{'https://nypost.com/2023/03/08/an-egregious-denial-of-due-process-for-jan-6-protesters/': ['nypost.com', 'An egregious denial of due process for Jan. 6 protesters', 'Social Links For Miranda Devine', datetime.datetime(2023, 3, 9, 3, 18, 20), "For more than two years, while they controlled every lever of power in Washington, the Democratic Party and their media allies told a one-sided story about what happened at the Capitol on Jan. 6, 2021.\nFor more than two years, the surveillance footage from the 2021 Capitol riot, aired this week by Tucker Carlson, was kept from the American public by a Democratic Congress so a false narrative became cemented in the public consciousness.\nFor more than two years, footage that could have exonerated Jan. 6 defendants was kept from their legal teams.\nPeople are in prison because of that cover-up.\nTake the footage aired by Carlson Monday night of so-called “QAnon Shaman” Jacob Chansley being escorted around the Capitol by police officers who help hi

In [13]:
def get_sources(all_news):
    news_sources = {}
    for article in all_news:
        source = all_news[article][0]
        if source not in news_sources:
            news_sources[source] = 1
        else:
            news_sources[source] += 1
    return news_sources

check_sources = get_sources(all_news)

check_sources

{'nypost.com': 4,
 'washingtonpost.com': 4,
 'usatoday.com': 4,
 'niemanlab': 2,
 'newsweek.com': 6,
 'thehill.com': 4,
 'washingtontimes.com': 2,
 'reuters.com': 5,
 'thefederalist.com': 4,
 'dailywire.com': 4,
 'foxcom': 5,
 'huffpost.com': 1,
 'mediaite.com': 1,
 'wsj.com': 4,
 'bbc.com': 2,
 'nytimes.com': 2,
 'factcheck': 1,
 'ibtimes.com': 1,
 'cnbc.com': 1,
 'cnn.com': 5,
 'abcgo.com': 2,
 'breitbart.com': 2,
 'townhall.com': 2,
 'axios.com': 3,
 'salon.com': 1,
 'dailycaller.com': 1,
 'business.com': 2,
 'apcom': 2,
 'www1.cbn.com': 1,
 'theatlantic.com': 1,
 'cbscom': 2,
 'thedispatch.com': 1,
 'csmonitor.com': 3,
 'washingtonexaminer.com': 1,
 'theverge.com': 1,
 'theepochtimes.com': 3,
 'theguardian.com': 2,
 'thepostmillennial.com': 1,
 'npr': 1,
 'motherjones.com': 1,
 'politico.com': 1,
 'msnbc.com': 1,
 'slate.com': 1}

In [ ]:
print(all_news)

{'https://nypost.com/2023/03/08/an-egregious-denial-of-due-process-for-jan-6-protesters/': ['nypost.com', 'An egregious denial of due process for Jan. 6 protesters', 'Social Links For Miranda Devine', datetime.datetime(2023, 3, 9, 3, 18, 20), "For more than two years, while they controlled every lever of power in Washington, the Democratic Party and their media allies told a one-sided story about what happened at the Capitol on Jan. 6, 2021.\nFor more than two years, the surveillance footage from the 2021 Capitol riot, aired this week by Tucker Carlson, was kept from the American public by a Democratic Congress so a false narrative became cemented in the public consciousness.\nFor more than two years, footage that could have exonerated Jan. 6 defendants was kept from their legal teams.\nPeople are in prison because of that cover-up.\nTake the footage aired by Carlson Monday night of so-called “QAnon Shaman” Jacob Chansley being escorted around the Capitol by police officers who help hi

In [26]:
data['main_headline'] = data['Source_Link'].apply(lambda x: all_news[x][1] if x in all_news else None)
data['text'] = data['Source_Link'].apply(lambda x: all_news[x][4] if x in all_news else None)
data['author'] = data['Source_Link'].apply(lambda x: all_news[x][2] if x in all_news else None)

In [27]:
data.head()

,Date,Description,Source_Name,Source_Bias,Source_Headline,Source_Link,main_headline,text,author
0,2023-03-09T08:06:49-08:00,"For more than two years, while they controlled...",OPINION: Trump’s risky call for protests,Left,https://www.allsides.com/news/2023-03-09-0806/...,https://nypost.com/2023/03/08/an-egregious-den...,An egregious denial of due process for Jan. 6 ...,"For more than two years, while they controlled...",Social Links For Miranda Devine
1,2020-11-04T21:50:01-08:00,When Donald Trump narrowly won Wisconsin in 20...,OPINION: Trump’s risky call for protests,Left,https://www.allsides.com/news/2020-11-04-2150/...,https://www.washingtonpost.com/politics/americ...,Election reveals deeper divides between red an...,The Washington Post’s Philip Bump analyzes how...,"Philip Rucker, Robert Costa"
2,2022-07-20T12:44:42-07:00,Joe Biden has won the U.S. presidential electi...,OPINION: Donald Trump has committed a lot of s...,Lean Left,https://www.allsides.com/news/2020-11-10-1027/...,https://www.niemanlab.org/2020/11/five-types-o...,Five types of misinformation to watch out for ...,Joe Biden has won the U.S. presidential electi...,"Kate Starbird, Jevin West, Renee Diresta, Josh..."
3,2020-12-16T11:02:25-08:00,WASHINGTON – Congressional Republicans began t...,Trump Indictment Could Be a 2024 Cash Cow,Center,https://www.allsides.com/news/2020-12-16-1101/...,https://www.usatoday.com/story/news/politics/e...,Republicans start to accept President-elect Biden,Republican senators slowly begin to accept Bid...,"Nicholas Wu, Ledyard King"
4,2020-11-09T19:10:07-08:00,U.S. Attorney General William Barr has authori...,Police surround NY Courthouse and DC Capitol i...,Center,https://www.allsides.com/news/2020-11-09-1910/...,https://www.newsweek.com/barr-tells-federal-pr...,Barr Tells Federal Prosecutors to Investigate ...,U.S. Attorney General William Barr has authori...,"Katherine Fung, Jeff Charles"


In [28]:
from urllib.parse import urlparse

def extract_domain(url):
    parsed_url = urlparse(url)
    domain = parsed_url.hostname.split('.')[-2]
    return domain

In [29]:
data['Source_Headline'] = data['Source_Link'].apply(extract_domain)
data = data.rename(columns={'Date': 'data', 'Description': 'description', 'Source_Name': 'source name', 'Source_Bias': 'bias', 
                            'Source_Headline': 'source', 'Source_Link': 'link'})
data.head()

,data,description,source name,bias,source,link,main_headline,text,author
0,2023-03-09T08:06:49-08:00,"For more than two years, while they controlled...",OPINION: Trump’s risky call for protests,Left,nypost,https://nypost.com/2023/03/08/an-egregious-den...,An egregious denial of due process for Jan. 6 ...,"For more than two years, while they controlled...",Social Links For Miranda Devine
1,2020-11-04T21:50:01-08:00,When Donald Trump narrowly won Wisconsin in 20...,OPINION: Trump’s risky call for protests,Left,washingtonpost,https://www.washingtonpost.com/politics/americ...,Election reveals deeper divides between red an...,The Washington Post’s Philip Bump analyzes how...,"Philip Rucker, Robert Costa"
2,2022-07-20T12:44:42-07:00,Joe Biden has won the U.S. presidential electi...,OPINION: Donald Trump has committed a lot of s...,Lean Left,niemanlab,https://www.niemanlab.org/2020/11/five-types-o...,Five types of misinformation to watch out for ...,Joe Biden has won the U.S. presidential electi...,"Kate Starbird, Jevin West, Renee Diresta, Josh..."
3,2020-12-16T11:02:25-08:00,WASHINGTON – Congressional Republicans began t...,Trump Indictment Could Be a 2024 Cash Cow,Center,usatoday,https://www.usatoday.com/story/news/politics/e...,Republicans start to accept President-elect Biden,Republican senators slowly begin to accept Bid...,"Nicholas Wu, Ledyard King"
4,2020-11-09T19:10:07-08:00,U.S. Attorney General William Barr has authori...,Police surround NY Courthouse and DC Capitol i...,Center,newsweek,https://www.newsweek.com/barr-tells-federal-pr...,Barr Tells Federal Prosecutors to Investigate ...,U.S. Attorney General William Barr has authori...,"Katherine Fung, Jeff Charles"


In [30]:
data.dropna(inplace = True)
data.reset_index(drop = True)

,data,description,source name,bias,source,link,main_headline,text,author
0,2023-03-09T08:06:49-08:00,"For more than two years, while they controlled...",OPINION: Trump’s risky call for protests,Left,nypost,https://nypost.com/2023/03/08/an-egregious-den...,An egregious denial of due process for Jan. 6 ...,"For more than two years, while they controlled...",Social Links For Miranda Devine
1,2020-11-04T21:50:01-08:00,When Donald Trump narrowly won Wisconsin in 20...,OPINION: Trump’s risky call for protests,Left,washingtonpost,https://www.washingtonpost.com/politics/americ...,Election reveals deeper divides between red an...,The Washington Post’s Philip Bump analyzes how...,"Philip Rucker, Robert Costa"
2,2022-07-20T12:44:42-07:00,Joe Biden has won the U.S. presidential electi...,OPINION: Donald Trump has committed a lot of s...,Lean Left,niemanlab,https://www.niemanlab.org/2020/11/five-types-o...,Five types of misinformation to watch out for ...,Joe Biden has won the U.S. presidential electi...,"Kate Starbird, Jevin West, Renee Diresta, Josh..."
3,2020-12-16T11:02:25-08:00,WASHINGTON – Congressional Republicans began t...,Trump Indictment Could Be a 2024 Cash Cow,Center,usatoday,https://www.usatoday.com/story/news/politics/e...,Republicans start to accept President-elect Biden,Republican senators slowly begin to accept Bid...,"Nicholas Wu, Ledyard King"
4,2020-11-09T19:10:07-08:00,U.S. Attorney General William Barr has authori...,Police surround NY Courthouse and DC Capitol i...,Center,newsweek,https://www.newsweek.com/barr-tells-federal-pr...,Barr Tells Federal Prosecutors to Investigate ...,U.S. Attorney General William Barr has authori...,"Katherine Fung, Jeff Charles"
...,...,...,...,...,...,...,...,...,...
93,2023-03-23T06:04:25-07:00,Former President Donald Trump knows his base b...,ANALYSIS: Jan. 6 panel holds prime-time televi...,Left,cnn,https://www.cnn.com/2023/03/19/opinions/trump-...,Opinion: Trump's risky call for protests,"Editor’s Note: Dean Obeidallah, a former attor...",Dean Obeidallah
94,2021-11-11T05:46:54-08:00,Donald Trump and his legal team haven't just p...,Trump aides' words to take center stage as U.S...,Center,msnbc,https://www.msnbc.com/rachel-maddow-show/how-f...,How far will Trump go to hide Jan. 6 documents...,Donald Trump and his legal team haven't just p...,"Https, Media-Cldnry.S-Nbcnews.Com Image Upload..."
95,2020-10-16T17:11:03-07:00,Several members within President Donald Trump'...,OPINION: A Key Reason Jan. 6 Rioters Aren’t Fa...,Left,newsweek,https://www.newsweek.com/trumps-orbit-now-has-...,Trump's Orbit Now Has More New COVID Cases Tha...,Several members within President Donald Trump'...,"Jocelyn Grzeszczak, Jeff Charles"
96,2022-08-11T07:48:32-07:00,The raid on Mar-a-Lago was based largely on in...,OPINION: How to Stop Trump and Prevent Another...,Lean Left,newsweek,https://www.newsweek.com/exclusive-informer-to...,Exclusive: An Informer Told the FBI What Docs ...,The raid on Mar-a-Lago was based largely on in...,"William M. Arkin, Jeff Charles"


In [ ]:
data.info()

Now we have more complete information. However, we still want to clean it up a bit. We're going to start by dropping all of the rows with missing stories (or mixed stories). After that, we're going to drop the stories that are less than 250 characters and greater than 20,000 characters.

Afterward, we'll save the results to a CSV for further analysis later on.

In [31]:
data = data.drop(data[data.text == 'None'].index)
data = data.drop(data[data.text == 'Mixed'].index)

data['text_len'] = data.apply(lambda row: len(row.text), axis=1)

data = data.drop(data[data.text_len > 20000].index)
data = data.drop(data[data.text_len < 250].index)

# write to csv
data.to_csv('news-corpus-df.csv', index=False)
data.head()


,data,description,source name,bias,source,link,main_headline,text,author,text_len
0,2023-03-09T08:06:49-08:00,"For more than two years, while they controlled...",OPINION: Trump’s risky call for protests,Left,nypost,https://nypost.com/2023/03/08/an-egregious-den...,An egregious denial of due process for Jan. 6 ...,"For more than two years, while they controlled...",Social Links For Miranda Devine,6875
1,2020-11-04T21:50:01-08:00,When Donald Trump narrowly won Wisconsin in 20...,OPINION: Trump’s risky call for protests,Left,washingtonpost,https://www.washingtonpost.com/politics/americ...,Election reveals deeper divides between red an...,The Washington Post’s Philip Bump analyzes how...,"Philip Rucker, Robert Costa",9792
2,2022-07-20T12:44:42-07:00,Joe Biden has won the U.S. presidential electi...,OPINION: Donald Trump has committed a lot of s...,Lean Left,niemanlab,https://www.niemanlab.org/2020/11/five-types-o...,Five types of misinformation to watch out for ...,Joe Biden has won the U.S. presidential electi...,"Kate Starbird, Jevin West, Renee Diresta, Josh...",8074
3,2020-12-16T11:02:25-08:00,WASHINGTON – Congressional Republicans began t...,Trump Indictment Could Be a 2024 Cash Cow,Center,usatoday,https://www.usatoday.com/story/news/politics/e...,Republicans start to accept President-elect Biden,Republican senators slowly begin to accept Bid...,"Nicholas Wu, Ledyard King",3540
4,2020-11-09T19:10:07-08:00,U.S. Attorney General William Barr has authori...,Police surround NY Courthouse and DC Capitol i...,Center,newsweek,https://www.newsweek.com/barr-tells-federal-pr...,Barr Tells Federal Prosecutors to Investigate ...,U.S. Attorney General William Barr has authori...,"Katherine Fung, Jeff Charles",3369


In [32]:
bybias = data.groupby('bias')
bybias['text_len'].describe()

,count,mean,std,min,25%,50%,75%,max
bias,,,,,,,,
Center,26.0,5450.000000,3185.381070,398.0,3776.50,4136.0,6280.25,13480.0
Lean Left,20.0,5817.150000,3682.108753,624.0,2174.25,5194.0,8293.00,12413.0
Lean Right,6.0,7356.833333,4005.722328,2532.0,3994.25,7699.0,10452.00,12082.0
Left,14.0,4158.071429,2589.776957,252.0,2290.25,3820.5,6123.50,9792.0
Right,29.0,5788.827586,4763.016819,320.0,2418.00,4330.0,6503.00,19419.0
